<a href="https://colab.research.google.com/github/alex-jk/painting-lora-finetune/blob/main/neural_style_transfer_photos_to_paintings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# set before importing torch; then Runtime -> Restart runtime
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

CONTENT = "/content/drive/MyDrive/nst/images/content.jpg"
STYLE   = "/content/drive/MyDrive/nst/images/style.jpg"

# Option to load generated images from Google Drive
generated_out_path = "/content/drive/MyDrive/nst/images/out.png"
generated_out_painterly_path = "/content/drive/MyDrive/nst/images/out_painterly.png"

get_generated_images_from_drive = False

<h3><font color="#0b3d91">Neural Style Transfer Method</font></h3>

**Goal**

Turn your photo into a painting-like image without moving objects: keep the photo’s layout, borrow the painting’s colors/texture.

**How it works (high level)**<br>
We create an output image `X` and optimize its pixels so that:<br>
• its **deep features** (from a CNN) match the photo → preserves structure/content;<br>
• its **feature statistics** (Gram matrices) match the painting → transfers style/brushwork.

**VGG-19**<br>
**VGG-19** is a classic **convolutional neural network (CNN)** trained on `ImageNet`. It stacks many conv layers that detect edges, textures, parts, and objects.<br>
• **Shallow layers** respond to color/texture; **deep layers** to object/layout.<br>
• Freeze VGG (no training) and use it only to extract features that act as "perceptual rulers."<br>
• To preserve the photo's content, we match its core VGG features. To inject the painting's style, we match its Gram stats, which are a statistical measurement of which textures and patterns (like certain brushstrokes and colors) tend to appear together throughout the image.

<h3><font color="#0b3d91">How the neural style transfer model is trained (fast/feed-forward version)</font></h3>

**Notation**
- `f_phi`: trainable **stylizer** `CNN` (the only network you update)
- `VGG`: frozen **loss network** (used only to compute losses)
- `C`: content image, `S`: style image
- `X = f_phi(C)`: stylized output

**Training loop**
1. **Freeze VGG.** Use it only to extract features for the losses (no weight updates).
2. **Choose a style** image `S` (or sample from a style set).
3. **For each content image `C`:**
    - **Forward Pass:** We feed the content photo `C` through our trainable stylizer CNN `f_phi` to produce a candidate image `X`. This network is the one we are actively training, and its output is our `stylized guess`, which we will immediately use to measure how well we are matching our style and content goals.
    - **Extract features (with frozen VGG):** get feature maps `F_l(C)`, `F_l(X)`, and `F_l(S)` from selected layers `l`.
    - **Content loss:** This is the penalty for changing the photo's core content. We compute it by taking the feature representations of our stylized image `F_l(X)` and the original photo `F_l(C)` from a deeper VGG layer and calculating the squared Euclidean distance (or Mean Squared Error) between them. A smaller distance means the main objects in our guess still look like the original.
    - **Style loss:** make **Gram matrices** of `X` match those of `S` across chosen layers (captures texture/brushwork).
    - **Total Variation loss (optional):** This is a penalty added to make the final image smoother and less noisy. The actual penalty is the `Total Variation` of the image `X`, which is calculated by summing the differences between each pixel and its immediate neighbors, both horizontally and vertically. A high value means the image has a lot of sharp, disconnected pixel noise, and minimizing it creates more gradual transitions.
    - **Total loss:** `L = lambda_c * L_content + lambda_s * L_style + lambda_tv * L_tv`.
    - **Backprop:** send gradients through VGG into `f_phi` (VGG stays frozen) and **update only `phi`** (e.g., Adam). *(During training, only change the weights of the stylizer network (f_phi). Do not change the weights of the VGG loss network.)*
4. **Repeat** over batches until validation loss plateaus. The trained `f_phi` then stylizes any new `C` in a single forward pass.

> **Alternative (optimization-based NST):** don’t learn `f_phi`. Initialize `X` (e.g., `X <- C`) and **optimize the pixels of `X`** directly to minimize the same losses with VGG frozen.


**Pipeline**<br>
Load & normalize images → run through frozen `VGG` → compute `content loss` (deep layer), `style loss` (Gram matrices across several layers), plus `total variation (TV) smoothing` → then, in an optimization loop, backprop gradients to the image pixels with L-BFGS/Adam. This process is repeated for hundreds of steps, progressively modifying the image to find the pixels that result in the `lowest possible total loss score`.<br>

**Total Variation (TV) smoothing** - A regularizer that penalizes rapid pixel-to-pixel changes. It prefers images that are locally smooth (piecewise-smooth) without simply blurring everything.

<h3><font color="#0b3d91">Gram matrices (style loss)</font></h3>

**Setup (for one VGG layer `l`)**

To transfer an artistic style, we need a way to measure 'style' itself. A simple feature match isn't enough, because we don't want to copy the objects from the style painting, just its texture and feel. Gram matrices are the solution. They are a mathematical tool that describes an image's style by ignoring object placement and instead capturing which visual patterns (like colors and brushstrokes) tend to appear together. By forcing our new image to have the same Gram matrix as the style painting, we force it to adopt the same artistic signature.

- **Get Feature Maps from an Image:**
First, we need to extract visual features from an image. Let's take our current version of the output image `X` as the example. We feed it through a specific layer `l` of the `VGG network`.
The output, `F_l(X)`, is a set of "feature maps": This is a 3D block of data with shape `[C, H, W]`, representing what the VGG layer "sees" in our generated image.
- **Flatten the Feature Maps:**
The next step is to prepare this 3D block of data for the style calculation by collapsing its spatial dimensions `the height H and width W`. We reshape the feature maps `F_l(X)` into a new 2D matrix, `F`.
  - This `F` matrix now has a shape of `[C, H*W]`. Each of the `C` rows represents a different feature channel, and the columns represent all the pixel locations. This transformation is essential because it organizes the data so we can easily measure the relationships between the different feature channels, which is the key to capturing style.
- **Gram matrix (per layer):**
  - Start with features `F_l(X)` of shape `[C, H, W]`.
  - Flatten spatial dims → `F` has shape `[C, N]`, where `N = H * W`.
  - Indices: `i, j ∈ {1..C}` are **channel indices**; `p ∈ {1..N}` is the **pixel (spatial) index** after flattening.
  - Define the `[C, C]` matrix `G` by  
    `G[i, j] = (1/N) * sum_{p=1..N} F[i, p] * F[j, p]`.
  - This is the **average product across all spatial positions** of channels `i` and `j`.  
    Averaging over `p` removes location and keeps **co-activation (texture) statistics**.

**Intuition**
- Entry `G[i, j]` is the dot product between channel `i` and `j` across all pixels → how strongly they **co-activate**.
- Averaging over locations discards exact positions, so Gram matrices capture **texture/brushwork statistics** (which features occur together), not layout.
- Matching `G_l(X)` to `G_l(S)` makes `X` use the same color/texture patterns as the style image at that layer’s scale.

**Why multiple layers?**
- Shallow layers → fine textures/colors; deeper layers → broader patterns.
- Summing style losses over several layers gives a **multi-scale** style match.

**Style loss formula**

The core of the style loss is a direct comparison between two Gram matrices at each VGG layer `l`:
  - One matrix, `G_l(S)`, is computed from the original style painting `S`. This is our fixed target; it represents the "correct" artistic style for that layer.
  - The other matrix, `G_l(X)`, is computed from our current generated image `X`. This represents the style of our work-in-progress.

The goal is to change our generated image `X` until its Gram matrix `G_l(X)` becomes as close as possible to the target `G_l(S)`.
- Per layer: `L_style_l = || G_l(X) - G_l(S) ||_F^2`
- Total style loss: weighted sum over chosen layers.
- Normalize by `H*W` (or `C*H*W`) to keep values scale-stable.

**Install dependencies**

In [ ]:
import torch, torchvision, PIL
print("Torch:", torch.__version__, "| Vision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())

In [ ]:
# ===================== SETUP =====================
import math, os, time
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
from pathlib import Path

#### **Set device and ImageNet normalization (for VGG-19 features)**

- **Device selection:** Use GPU (`cuda`) if available; otherwise fall back to CPU. All tensors/ops must be on the **same device**.
- **ImageNet normalization:** VGG-19 expects RGB inputs scaled to **[0,1]** and normalized per channel with:
  - mean = `[0.485, 0.456, 0.406]`
  - std  = `[0.229, 0.224, 0.225]`
<br>We apply it as: `(x - mean) / std` (broadcast per channel over H×W).
- **Why:** Feeding the exact normalization used in training keeps VGG feature distributions correct; skipping it can cause unstable optimization or odd colors.
- **`.to(device)` on mean/std:** Puts these constants on the same device as your images to avoid device-mismatch errors.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406], device=device)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225], device=device)

###**NST Code**

#### Image I/O helpers

**`load_image(path, target_long_side=None)`**
- **Force RGB (3 channels):** keeps shape consistent for VGG / PyTorch.
- **Optional shrink (keep aspect ratio):** if `target_long_side` is set, scale the image so its **longer side = target_long_side**.  
  *Why:* fewer pixels ⇒ faster, uses less memory. (Halving width & height ≈ **4× fewer pixels**.)
- **High-quality resize (`Image.LANCZOS`):** when you shrink, many pixels are merged. A good filter **averages smartly** so diagonals don’t look like **stair-steps (“jaggies”)** and fine textures don’t turn into **wavy stripes**. Cleaner inputs ⇒ cleaner VGG features.
- **To tensor in `[0,1]`:** `transforms.ToTensor()` gives a float tensor **(3, H, W)** scaled to `[0,1]`.
- **Add batch dim:** `unsqueeze(0)` → **(1, 3, H, W)** because models expect a batch.
- **Send to device:** `.to(device)` moves it to CPU or GPU.

**`save_image(tensor, path)`**
- **Stop tracking gradients:** `detach()` — we’re just saving numbers now.
- **Keep valid range:** `clamp(0,1)` so colors aren’t out of bounds.
- **Back to CPU & drop batch:** `cpu().squeeze(0)` → **(3, H, W)**.
- **Write file:** `transforms.ToPILImage()(x).save(Path(path))`.

In [ ]:
def load_image(path, target_long_side=None):
    # Open image from disk and ensure it has 3 channels (Red, Green, Blue)
    img = Image.open(path).convert("RGB")

    if target_long_side is not None:
        w, h = img.size
        # Calculate the scaling factor so the longest side equals 'target_long_side'
        s = target_long_side / max(w, h)
        # Resize the image using Lanczos resampling: a windowed-sinc interpolation. Lanczos is a high-quality method for shrinking images.
        # Instead of taking just a few nearby pixels (like bilinear or bicubic), it looks at a larger area around each pixel and averages them in a smart way.
        # The result: sharper details and fewer blur artifacts when the image is reduced.
        img = img.resize((round(w*s), round(h*s)), Image.LANCZOS)

    # Transform pipeline:
    # 1. transforms.ToTensor(): Converts PIL Image (H, W, C) range [0, 255] to Torch Tensor (C, H, W) range [0.0, 1.0]
    # 2. unsqueeze(0): Adds a batch dimension at index 0 -> Shape becomes [1, 3, H, W]
    # 3. to(device): Moves the tensor to the global 'device' (e.g., GPU/CUDA)
    x = transforms.ToTensor()(img).unsqueeze(0).to(device)
    return x

def save_image(tensor, path):
    # Post-process tensor for saving:
    # 1. detach(): Removes tensor from the computational graph (stops gradient tracking)
    # 2. clamp(0, 1): Clips values to ensure they are valid colors (fixes potential oversaturation)
    # 3. cpu(): Moves data back to system RAM (PIL cannot save from GPU)
    # 4. squeeze(0): Removes the batch dimension -> Shape becomes [3, H, W]
    x = tensor.detach().clamp(0,1).cpu().squeeze(0)

    # Convert the Tensor back to a PIL image (automatically scales 0.0-1.0 to 0-255)
    # and save it to the specified filesystem path
    transforms.ToPILImage()(x).save(Path(path))

#### Normalization module

**What it does**
- Takes an image batch `x` with shape **(N, 3, H, W)** in **[0,1]** and applies **per-channel** ImageNet normalization:  
  `x_norm = (x − mean) / std`.
- Stores `mean` and `std` as tensors shaped **(1, 3, 1, 1)** so they **broadcast** across batch and spatial dims (H×W).

**Why we need it**
- **VGG-19** was trained on ImageNet-normalized RGB. Feeding inputs in the same scale makes VGG features **comparable and stable** for style/content losses.
- Using `register_buffer(...)` puts `mean/std` on the **right device**, includes them in the **state_dict**, and keeps them **non-trainable** (not updated by the optimizer).

*Typical constants:* `mean = [0.485, 0.456, 0.406]`, `std = [0.229, 0.224, 0.225]`.

In [ ]:
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        # mean and std are 3-element tensors: [R, G, B].
        # We reshape them to [1, 3, 1, 1] so PyTorch can automatically apply the same value to every pixel
        # in that channel for any image size or batch size.
        self.register_buffer("mean", mean.view(1, 3, 1, 1))
        self.register_buffer("std",  std.view(1, 3, 1, 1))

        # register_buffer:
        # - These values are part of the module (they get saved with it), but they are NOT trainable (no gradients).
        # - When you move the model to GPU/CPU with .to(device), these values move with it automatically.

    def forward(self, x):
        # x is an image tensor with shape [batch, 3, H, W].
        # For each color channel (R, G, B), subtract the mean and divide by the std. This puts pixel values on the same scale as the images used to train VGG.
        return (x - self.mean) / self.std

### VGG “taps” — which layers we read and why

**What is a tap?**  
A *tap* is a layer in VGG where we **read** the activation (feature map) during the forward pass. VGG is **frozen** (not trained). We only use the tapped layers in our losses; **all other layers are ignored**.

**Are we using all layers?**  
**No.** We select a small subset:
- **Content taps** → preserve the photo’s layout/structure.
- **Style taps** → capture texture/brush/color statistics at multiple scales.

**Our choices**
```python
# human-readable names for vgg19.features indices
VGG_LAYER_NAMES = {1:"relu1_1", 6:"relu2_1", 11:"relu3_1", 20:"relu4_1", 22:"relu4_2", 29:"relu5_1"}

# use this deep layer to keep object/layout structure
CONTENT_LAYERS = ["relu4_2"]

# use these (shallow→deep) to capture style across scales
STYLE_LAYERS   = ["relu1_1","relu2_1","relu3_1","relu4_1","relu5_1"]

In [ ]:
# VGG layer taps
VGG_LAYER_NAMES = {1:"relu1_1", 6:"relu2_1", 11:"relu3_1", 20:"relu4_1", 22:"relu4_2", 29:"relu5_1"}
CONTENT_LAYERS = ["relu4_2"]
STYLE_LAYERS   = ["relu1_1","relu2_1","relu3_1","relu4_1","relu5_1"]

### `VGGFeatures` — frozen VGG wrapper that returns only the layers we care about

**Goal:** In one forward pass, (1) normalize the input exactly like ImageNet and (2) collect activations at selected “tap” layers for our losses.

**How it’s built (`__init__`):**
- Load `vgg19.features` (try the new weights API; fall back to `pretrained=True`).
- Call `eval()` and set `requires_grad_(False)` → VGG is **frozen** (we never train it).
- Create `Normalization(mean, std)` so inputs are ImageNet-normalized before entering VGG.

**What `forward(x)` does:**
1. Normalize: `x ← (x − mean) / std`.
2. Run through VGG **layer by layer**.
3. If the current layer index is listed in `VGG_LAYER_NAMES`, **store that activation** under its readable name (e.g., `"relu4_2"`).
4. Return a dict mapping names → activations (each `(N, C, H, W)`), e.g.:
   `{"relu1_1": T1, "relu2_1": T2, "relu3_1": T3, "relu4_1": T4, "relu4_2": T5, "relu5_1": T6}`.

**Why this design:**
- We do **not** use all VGG layers. We tap a small subset:
  - **Content loss:** compare `X` vs `C` at a deep tap (e.g., `"relu4_2"`).
  - **Style loss:** compare Gram stats at several shallow→deep taps.
- Freezing VGG + correct normalization → features are **stable and comparable**; only your stylizer (or pixels) is updated.

In [ ]:
class VGGFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        # Load the convolutional part of a pretrained VGG19 network. We use built-in ImageNet weights (IMAGENET1K_V1).
        feats = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features

        # Turn off "inplace" for all ReLU layers. This makes PyTorch keep the input and output of ReLU separate,
        # which is safer when we need gradients for style transfer.
        for m in feats.modules():
            if isinstance(m, nn.ReLU):
                m.inplace = False

        # Put VGG in evaluation mode (no dropout, no batch-norm updates) and move it to the chosen device (CPU/GPU).
        self.vgg = feats.eval().to(device)

        # Freeze all VGG weights: we never train VGG itself.
        for p in self.vgg.parameters():
            p.requires_grad_(False)

        # Normalization layer so input images match the statistics (mean and std) of the images VGG was trained on.
        self.norm = Normalization(IMAGENET_MEAN, IMAGENET_STD)

    def forward(self, x):
        # Dictionary to store the feature maps we care about.
        out = {}

        # First normalize the input image.
        x = self.norm(x)

        # Pass through each layer of VGG in order.
        for i, layer in enumerate(self.vgg):
            x = layer(x)

            # If this layer index is one we care about, save its output under a readable name like "conv1_1", "conv2_1", etc.
            if i in VGG_LAYER_NAMES:
                out[VGG_LAYER_NAMES[i]] = x

        # Return a dict: {layer_name: feature_tensor}
        return out


def gram(feat):  # Gram in fp32 for stability
    # Convert to float32 to avoid numeric issues in the matrix multiply.
    feat = feat.float()

    # feat has shape [batch, channels, height, width]
    _, C, H, W = feat.shape

    # Flatten the spatial dimensions so each row is one channel:
    # shape becomes [C, H*W].
    Fm = feat.view(C, H * W)

    # Compute the Gram matrix: [C, C].
    # Each entry measures how strongly two channels tend to be "on" together.
    # We divide by the number of pixels so the result doesn't depend on image size.
    return (Fm @ Fm.t()) / (H * W)


### Precompute content & style targets — what this code does

1. **Load images** (`C`, `S`) at a manageable size → tensors `(1,3,H,W)` on the correct device.
2. **Frozen VGG taps**: build `VGGFeatures()` (it normalizes to ImageNet and returns only the tapped layers).
3. **Forward once**: run `C` and `S` through VGG → get dictionaries of tapped activations: `c_feats`, `s_feats`.
4. **Content target(s)**: take the content image activations at `CONTENT_LAYERS` (e.g., `relu4_2`). These preserve layout/structure.
5. **Style targets**: for each layer in `STYLE_LAYERS`, compute a **Gram matrix** from the style activations:
   - reshape features to `(C, N)` where `C`=channels and `N=H*W`;
   - build a `(C×C)` matrix of **average channel-wise products** → captures texture/color statistics while ignoring exact positions.
   - **Why Gram matrices at multiple style layers?**  
Style is about which features co-occur, not where. A Gram matrix captures channel co-activation (second-order stats) and ignores spatial layout, so matching `G_l(X)` to `G_l(S)` transfers texture/brushwork without copying positions. Using several layers (shallow→deep) covers multi-scale style; normalizing by `H×W` keeps losses comparable across sizes.

6. **Sanity check**: print shapes of targets to confirm they were created.

**Use next:** feed `content_targets` and `style_targets` into your loss:
- (a) **Optimize pixels** `X` directly, or
- (b) **Train a stylizer network** `f_ϕ` so that `X = f_ϕ(C)`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

CONTENT = "/content/drive/MyDrive/nst/images/content.jpg"
STYLE   = "/content/drive/MyDrive/nst/images/style.jpg"

In [ ]:
def total_variation(x):
    """
    Total variation (TV) loss.

    What it does:
    - Measures how "noisy" or "jagged" an image is.
    - It adds up the absolute differences between neighboring pixels (up/down and left/right) and averages them.
    - A lower value means smoother images.

    How it’s used:
    - In style transfer, this is usually applied ONLY to the **generated image**
      (the image we are optimizing), not to the content or style reference images.
    - It is added as an extra term in the loss function to encourage the
      generated image to be smoother and less pixel-noisy.

    x: Tensor of shape [batch, 3, H, W].
    """
    # Vertical differences: compare each pixel to the pixel below it
    tv_h = (x[:, :, 1:, :] - x[:, :, :-1, :]).abs().mean()

    # Horizontal differences: compare each pixel to the pixel to the right
    tv_w = (x[:, :, :, 1:] - x[:, :, :, :-1]).abs().mean()

    # Total variation is the sum of vertical and horizontal averages
    return tv_h + tv_w

In [ ]:
if not get_generated_images_from_drive:
    # We will work with a smaller version of both images so optimization is faster. The longest side of each image will be 512 pixels.
    target_long_side = 512

    # Load the content and style images as tensors on the correct device.
    C = load_image(CONTENT, target_long_side)
    S = load_image(STYLE,   target_long_side)

    # VGGFeatures wraps a pretrained VGG19 and returns feature maps from selected layers for style/content.
    vgg = VGGFeatures()

    # Extract feature maps for the fixed content and style reference images.
    # We don't need gradients for these (they never change), so we wrap
    # this in torch.no_grad() to save memory and speed.
    with torch.no_grad():
        c_feats = vgg(C)
        s_feats = vgg(S)

    # Store the "target" content features for the chosen content layers.
    # These are what we want the generated image to match in terms of content.
    content_targets = {n: c_feats[n].detach() for n in CONTENT_LAYERS}

    # Store the "target" style Gram matrices for the chosen style layers.
    # These are what we want the generated image to match in terms of style.
    style_targets   = {n: gram(s_feats[n]).detach() for n in STYLE_LAYERS}

    print("Content taps:")
    for k in content_targets:
        print(" ", k, ":", tuple(content_targets[k].shape))
    print("Style Gram targets:")
    for k in STYLE_LAYERS:
        print(" ", k, ":", tuple(style_targets[k].shape))

    # Optimization settings:
    # - steps: number of gradient descent steps
    # - lr: learning rate for Adam
    # - w_content, w_style, w_tv: weights for each loss component
    steps, lr = 300, 0.02
    w_content, w_style, w_tv = 1.0, 10.0, 1e-5

    # X is the image we will actually optimize.
    # Start from the content image C (cloned so we don't modify C itself).
    # channels_last can give small speedups on some GPUs.
    X = C.clone().to(memory_format=torch.channels_last).requires_grad_(True)

    # Adam optimizer will update the pixels of X directly.
    opt = torch.optim.Adam([X], lr=lr)

    # GradScaler + autocast: mixed-precision for speed on GPU, with scaling
    # to avoid numerical issues. If no GPU is available, this effectively
    # does nothing (enabled=False).
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    for t in range(1, steps + 1):
        opt.zero_grad()

        # Run VGG forward in mixed precision (if on GPU) to speed things up.
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            feats = vgg(X)

        # Content loss: how different the generated image features are
        # from the content target features, on selected layers.
        Lc = sum(
            F.mse_loss(feats[n].float(), content_targets[n].float())
            for n in CONTENT_LAYERS
        )

        # Style loss: difference between Gram matrices of the generated image
        # and the style targets, on selected layers.
        Ls = 0.0
        for n in STYLE_LAYERS:
            Ls += F.mse_loss(gram(feats[n]), style_targets[n].float())

        # Total variation loss: encourages smoothness in the generated image.
        # This is applied ONLY to X (the image we’re optimizing).
        Ltv = total_variation(X.float())

        # Combine the three parts into the final loss.
        loss = w_content * Lc + w_style * Ls + w_tv * Ltv

        # Backpropagation with gradient scaling (for mixed precision).
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        # Keep pixel values in the valid range [0, 1] so the image
        # stays displayable throughout optimization.
        with torch.no_grad():
            X.clamp_(0, 1)

        # Progress logging every 50 steps and at the very end.
        if t % 50 == 0 or t == steps:
            print(
                f"[{t}/{steps}] "
                f"Lc={Lc.item():.4f}  "
                f"Ls={Ls.item():.4f}  "
                f"Tv={Ltv.item():.6f}  "
                f"Total={loss.item():.4f}"
            )

    # Save the final stylized image to disk.
    save_image(X, "out.png")
    print("Saved small stylized image -> out.png")

In [ ]:
from PIL import Image
if not get_generated_images_from_drive:
  display(Image.open("out.png"))

### Painterly stylization

**Goal**  
Create a stylized image that looks more like a painting: keep the overall composition while reducing fine photo detail and “sand” noise.

---

#### Hyperparameters
- `steps = 500` — number of optimization updates.
- `w_content = 1.0` — weight for **content loss** (keeps global structure).
- `w_style = 12.0` — weight for **style loss** (drives brush/texture/color).
- `w_tv = 5e-5` — weight for **total variation loss** (adds smoothness; suppresses speckle).

*Effect:* increase `w_style` → stronger style; increase `w_content` → more structure; increase `w_tv` → smoother, less noisy.

---

#### Style layers (which “style” features to match)
- We emphasize **mid/deep taps** (`relu3_1`, `relu4_1`) and down-weight **shallow taps** (`relu1_1`, `relu2_1`, `relu5_1`).
- *Why:* shallow layers tend to produce very fine, high-frequency textures; mid/deep taps encourage broader, painterly patterns.

---

#### Losses computed each step
- **Content loss (`Lc`)**: MSE between VGG features of the current image `X` and the content image at `CONTENT_LAYERS` → preserves overall arrangement.
- **Style loss (`Ls`)**: MSE between **Gram matrices** of `X` and the style image at `STYLE_LAYERS`, scaled by the per-layer weights → transfers multi-scale texture/brush/color statistics.
- **Total variation (`Ltv`)**: encourages neighboring pixels to be similar → reduces pixel-level noise.

---

#### Gaussian blur helper (`gauss_blur`)
- **Gaussian Blur:** An effect that smooths an image by applying a weighted average to each pixel based on its neighbors.
- **How it Works:** It uses a kernel, a small matrix of weights that slides over the image to define the blur. A Gaussian kernel's weights follow a bell curve, giving the most importance to the central pixel for a natural-looking effect.
- **Implementation:** This function builds a 1D kernel and applies it with two efficient, separable convolutions (vertical then horizontal).
- **Why used here:** By applying a very mild blur, it smooths out fine, pixel-level noise across the entire image while being too weak to significantly alter the large, coherent shapes.

---

#### Initialization
- Start `X` from a **mildly blurred** version of the content image plus **tiny noise**.
- *Why:* retains composition but softens sharp photo edges, making it easier to form paint-like regions rather than copying photographic detail.

---

#### Optimization loop (what happens each iteration)
1. Run `X` through **frozen VGG** to get tapped features.
2. Compute `Lc`, `Ls` (with the style layer weights), and `Ltv`.
3. Combine them into the total loss and use **Adam to update the pixels of `X`**.
4. Clamp `X` to `[0,1]` to keep valid image values.
5. Every 200 steps, apply a **very light blur** to reduce accumulating high-frequency noise.

---

#### Output
- Saves the final image to `/content/out_painterly.png`.

---

#### Quick tuning
- **Sharper / more structure:** slightly increase `w_content`, reduce or remove the periodic blur, or lower `w_tv`.
- **Looser / more painterly:** decrease `w_content`, increase `w_style`, or reduce shallow style layer weights further.
- **Less noise:** increase `w_tv` a bit or make the periodic blur slightly more frequent/stronger.

### Why there’s a Gaussian blur

**Purpose (what it’s for)**  
Lightly blur the working image `X` to suppress **tiny, high-frequency details** so optimization prefers **larger, paint-like regions** instead of gritty photo texture.

**What a kernel is (and why it’s odd-sized)**  
A *kernel* is a small sliding window of weights that averages each pixel with its neighbors.  
Odd sizes (3,5,7,…) give a single **center** that aligns with the pixel being updated.

**Gaussian blur = bell-curve averaging**  
- **Offsets `i`** = positions **relative to the pixel being updated**. If the kernel size is `k` (odd), let `r = (k-1)/2`. Then `i ∈ {-r, …, +r}` are the neighbors left/right (for a row) or up/down (for a column) of the **current pixel**.
- **Weights (bell curve):** for each offset `i`, compute an unnormalized weight  
  `w_tilde[i] = exp( - i^2 / (2 * sigma^2) )`.  
  Then **normalize** so they sum to 1 (keeps brightness):  
  `Z = sum_{j=-r}^{r} w_tilde[j]` and `w[i] = w_tilde[i] / Z`.
- **Applied to each pixel:** to blur one row at position `(m, n)` you replace the center value with a **weighted average of its neighbors**:  
  `X_hat[m, n] = sum_{i=-r}^{r} w[i] * X[m, n - i]`  (horizontal pass)  
  and similarly for a **vertical pass** over columns:  
  `Y[m, n] = sum_{i=-r}^{r} w[i] * X_hat[m - i, n]`.  
  (Two 1-D passes = full 2-D Gaussian blur, faster and equivalent.)
- **Intuition:** closer neighbors (small |i|) get larger weights → the pixel becomes a smooth average of its neighborhood, removing tiny high-frequency detail while preserving larger shapes.

**From 1-D to 2-D (separable, fast, same result)**  
2-D Gaussian kernel is the outer product: `K[i,j] = w[i] * w[j]`.  
Convolution can be done as two 1-D passes:  
• Vertical: `X_hat[m,n] = sum_{i=-r}^{r} w[i] * X[m-i, n]`  
• Horizontal: `Y[m,n] = sum_{j=-r}^{r} w[j] * X_hat[m, n-j]`

**How the function `gauss_blur` implements this**  
- Builds the 1-D Gaussian weights from `k` (kernel size) and `sigma` (spread) and **normalizes** them.  
- Applies **two depthwise convs**: vertical (`k×1`) then horizontal (`1×k`). *Depthwise* means each RGB channel is blurred **independently** (no color mixing).  
- Uses padding so the output stays the same size.

**How to tune (effect on sharpness)**  
- **Sharper (less blur):** smaller `sigma` (e.g., 0.6–1.0), smaller `k` (3–5), or call the blur less often.  
- **Softer (more blur):** larger `sigma`/`k`, or apply it more frequently.

**Bottom line**  
The Gaussian blur is a controlled low-pass filter: it dampens fine noise while keeping overall shapes, helping the style loss form coherent, painterly regions.

In [ ]:
if not get_generated_images_from_drive:
  # Painterly result: keep forms, suppress photo detail & high-frequency texture
  import torch
  import torch.nn.functional as F

  # --- weights
  steps = 500
  w_content = 1.0
  w_style  = 12.0
  w_tv     = 5e-4

  # --- emphasize mid/deep style layers (coarser patterns), de-emphasize shallow (fine noise)
  style_layer_weights = {
      "relu1_1": 0.1,
      "relu2_1": 0.2,
      "relu3_1": 1.2,
      "relu4_1": 1.0,
      "relu5_1": 0.4,
  }

  def total_variation(x):
      tv_h = (x[:, :, 1:, :] - x[:, :, :-1, :]).abs().mean()
      tv_w = (x[:, :, :, 1:] - x[:, :, :, :-1]).abs().mean()
      return tv_h + tv_w

  def gauss_blur(x, k=7, sigma=2.0):
      # cheap separable Gaussian using conv; expects (1,3,H,W)
      import math
      radius = k // 2
      xs = torch.arange(-radius, radius+1, device=x.device, dtype=x.dtype)
      w = torch.exp(-0.5 * (xs / sigma) ** 2)
      w = (w / w.sum()).view(1, 1, -1)                # (1,1,k)
      # depthwise conv along H then W
      x = F.conv2d(x, w.unsqueeze(3).expand(3,1,-1,1), padding=(radius,0), groups=3)
      x = F.conv2d(x, w.unsqueeze(2).expand(3,1,1,-1), padding=(0,radius), groups=3)
      return x

  # --- init: blurred content + tiny noise (keeps composition, kills crisp detail)
  with torch.no_grad():
    X0 = gauss_blur(C, k=5, sigma=1.0)     # milder blur
    X0 = (X0 + 0.005 * torch.randn_like(X0)).clamp(0, 1)  # less noise
  X = X0.clone().requires_grad_(True)

  opt = torch.optim.Adam([X], lr=0.02)

  for t in range(1, steps + 1):
      opt.zero_grad()

      feats = vgg(X)

      # content loss (layout)
      Lc = sum(F.mse_loss(feats[n], content_targets[n]) for n in CONTENT_LAYERS)

      # style loss (multi-scale, weighted)
      Ls = 0.0
      for n in STYLE_LAYERS:
          Ls = Ls + style_layer_weights[n] * F.mse_loss(gram(feats[n]), style_targets[n])

      # smoothness
      Ltv = total_variation(X)

      loss = w_content*Lc + w_style*Ls + w_tv*Ltv
      loss.backward()
      opt.step()

      with torch.no_grad():
          # occasional light blur suppresses new high-freq noise while preserving forms
          if t % 200 == 0:
              X.copy_(gauss_blur(X, k=3, sigma=0.8).clamp(0,1))
          X.clamp_(0, 1)

      if t % 50 == 0 or t == steps:
          print(f"[{t}/{steps}] Lc={Lc.item():.3f}  Ls={Ls.item():.3f}  Ltv={Ltv.item():.6f}  Total={loss.item():.3f}")

  save_image(X, "out_painterly.png")
  print("Saved: out_painterly.png")

In [ ]:
display(Image.open("out_painterly.png"))

<h3><font color="#0b3d91">Enlarge output image</font></h3>

In [ ]:
# ===================== UPSCALE + PER-TILE REFINE (memory-safe, rounding-safe) =====================
import math
import torch
import torch.nn.functional as F

def resize_long_side(img, long_side):
    _, _, h, w = img.shape
    if max(h, w) == long_side: return img
    if h >= w:
        new_h, new_w = long_side, int(round(long_side * w / h))
    else:
        new_w, new_h = long_side, int(round(long_side * h / w))
    return F.interpolate(img, size=(new_h, new_w), mode='bicubic', align_corners=False)

def make_blend_mask(H, W, overlap, device):
    m = torch.ones(1,1,H,W, device=device)
    for i in range(overlap):
        f = (i+1)/(overlap+1)
        m[:,:,i,:]    *= f
        m[:,:,-1-i,:] *= f
        m[:,:,:,i]    *= f
        m[:,:,:,-1-i] *= f
    return m

def gram32(feat):
    feat = feat.float()
    _, C, H, W = feat.shape
    Fm = feat.view(C, H*W)
    return (Fm @ Fm.t()) / (H*W)

# ---- 1) Pick your final size; 3000–4000 is plenty for most canvases ----
TARGET_LONG = 2400  # e.g. ~13×20" at 200–300 PPI depending on aspect
X_hi = resize_long_side(X.detach(), TARGET_LONG).clamp(0,1)

# ---- 2) Precompute low-res (512px canvas) content targets ONCE ----
with torch.no_grad():
    X_low_full = resize_long_side(X_hi, 512)
    c_low_all  = vgg(X_low_full)
content_targets_low = {n: c_low_all[n].detach() for n in CONTENT_LAYERS}

# ---- 3) Per-tile refine (bounded VRAM) ----
tile_size   = 640   # if OOM, drop to 576; if you have L4, try 768–896
overlap     = 96
tile_steps  = 80
tile_lr     = 0.01
wC, wS, wTV = 1.0, 30.0, 2e-5

H, W = X_hi.shape[2:]
stride = tile_size - overlap
blend_mask = make_blend_mask(tile_size, tile_size, overlap, X_hi.device)

final_num = torch.zeros_like(X_hi)
final_den = torch.zeros_like(X_hi)

print(f"Refining {W}x{H} with {tile_size}px tiles (overlap {overlap})...")

stride = tile_size - overlap
tiles_h = math.ceil(H / stride)
tiles_w = math.ceil(W / stride)
tiles_total = tiles_h * tiles_w
done = 0
print(f"Tiles: {tiles_h} × {tiles_w} = {tiles_total} total")

for i in range(0, H, stride):
    for j in range(0, W, stride):
        # tile coords on the big image
        i1 = min(i, H - tile_size)
        j1 = min(j, W - tile_size)
        i2, j2 = i1 + tile_size, j1 + tile_size

        # init tile from current canvas
        x_tile = X_hi[:, :, i1:i2, j1:j2].clone().requires_grad_(True)
        opt_t  = torch.optim.Adam([x_tile], lr=tile_lr)

        # ---------- SAFE: build content target crops in feature space ----------
        # map big tile -> 512-canvas pixel coords (use floor/ceil + clamp)
        sh_512 = 512 / H
        sw_512 = 512 / W
        ti1_512 = int(math.floor(i1 * sh_512))
        ti2_512 = int(math.ceil (i2 * sh_512))
        tj1_512 = int(math.floor(j1 * sw_512))
        tj2_512 = int(math.ceil (j2 * sw_512))
        ti1_512 = max(0, min(ti1_512, 512-1))
        tj1_512 = max(0, min(tj1_512, 512-1))
        ti2_512 = max(ti1_512+1, min(ti2_512, 512))
        tj2_512 = max(tj1_512+1, min(tj2_512, 512))

        content_targets_tile = {}
        for n in CONTENT_LAYERS:
            tgt_full = content_targets_low[n]               # e.g., relu4_2 on 512 canvas
            Hf, Wf = tgt_full.shape[2], tgt_full.shape[3]
            sh = Hf / X_low_full.shape[2]                   # per-layer scale
            sw = Wf / X_low_full.shape[3]
            ci1 = int(math.floor(ti1_512 * sh))
            ci2 = int(math.ceil (ti2_512 * sh))
            cj1 = int(math.floor(tj1_512 * sw))
            cj2 = int(math.ceil (tj2_512 * sw))
            ci1 = max(0, min(ci1, Hf-1))
            cj1 = max(0, min(cj1, Wf-1))
            ci2 = max(ci1+1, min(ci2, Hf))
            cj2 = max(cj1+1, min(cj2, Wf))
            content_targets_tile[n] = tgt_full[:, :, ci1:ci2, cj1:cj2]
        # ----------------------------------------------------------------------

        for t in range(1, tile_steps+1):
            opt_t.zero_grad()

            # content: downsample tile to the 512-canvas crop size (>=1x1)
            crop_h_512 = max(1, ti2_512 - ti1_512)
            crop_w_512 = max(1, tj2_512 - tj1_512)
            xt_low = F.interpolate(x_tile, size=(crop_h_512, crop_w_512),
                                   mode='bilinear', align_corners=False)
            feats_c = vgg(xt_low)

            Lc = 0.0
            for n in CONTENT_LAYERS:
                fn  = feats_c[n].float()
                tgt = content_targets_tile[n].float()
                th, tw = tgt.shape[2], tgt.shape[3]
                if fn.shape[2:] != (th, tw):  # rounding guard
                    fn = F.interpolate(fn, size=(max(1,th), max(1,tw)),
                                       mode='bilinear', align_corners=False)
                Lc += F.mse_loss(fn, tgt)

            # style: on the tile (uses your existing global style_targets)
            feats_s = vgg(x_tile)
            Ls = 0.0
            for n in STYLE_LAYERS:
                Ls += F.mse_loss(gram32(feats_s[n]), style_targets[n].float())

            Ltv = total_variation(x_tile.float())
            loss = wC*Lc + wS*Ls + wTV*Ltv
            loss.backward()
            opt_t.step()
            with torch.no_grad():
                x_tile.clamp_(0,1)

        # feather-blend tile back into the canvas
        with torch.no_grad():
            m = blend_mask[:, :, :x_tile.shape[2], :x_tile.shape[3]]
            final_num[:, :, i1:i2, j1:j2] += x_tile * m
            final_den[:, :, i1:i2, j1:j2] += m

        done += 1
        if torch.cuda.is_available():
            torch.cuda.synchronize()
            mem = torch.cuda.memory_allocated()/1e9
            print(f"✔ tile {done}/{tiles_total} | GPU {mem:.2f} GB")
        else:
            print(f"✔ tile {done}/{tiles_total}")

        del x_tile, opt_t, feats_c, feats_s, Lc, Ls, Ltv, loss
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

# ---- 4) Reconstruct and save ----
X_ref = (final_num / (final_den + 1e-8)).clamp(0,1)
save_image(X_ref, "/content/out_large.png")
print("Saved LARGE print image -> /content/out_large.png")

<h3><font color="#0b3d91">Smoothen out the image, add brush strokes</font></h3>

In [ ]:
# pip install -q opencv-contrib-python
import cv2, numpy as np

def oil_brush_balanced(
    in_path, out_path,
    brush_size=4, dyn_ratio=70, alpha=0.42,
    usm_radius=1.1, usm_amount=1.7,
    gamma=1.0,                  # 1.0 = no darkening; <1 brightens, >1 darkens
    pre_smooth=False,           # False keeps detail; True = bilateral pre-smooth
    add_edges=False, edge_weight=0.14, edge_low=90, edge_high=200
):
    img = cv2.imread(in_path, cv2.IMREAD_COLOR)
    if img is None: raise FileNotFoundError(in_path)

    base = cv2.bilateralFilter(img, 7, 30, 7) if pre_smooth else img

    # Oil-paint strokes
    oil = cv2.xphoto.oilPainting(base, brush_size, dyn_ratio)

    # Work in Lab (keep original a/b)
    lab_src = cv2.cvtColor(img,  cv2.COLOR_BGR2LAB)
    lab_oil = cv2.cvtColor(oil,  cv2.COLOR_BGR2LAB)
    Ls = lab_src[...,0].astype(np.float32)
    Lo = lab_oil[...,0].astype(np.float32)

    # Sharpen painted luminance
    blur = cv2.GaussianBlur(Lo, (0,0), usm_radius)
    Lo_sharp = (1.0 + usm_amount) * Lo - usm_amount * blur

    # Blend THEN tone-match to the original (prevents global darkening)
    Lmix = cv2.addWeighted(Lo_sharp, float(alpha), Ls, float(1.0 - alpha), 0.0, dtype=cv2.CV_32F)

    # --- replace the mean/std match with histogram matching ---
    def _hist_match_to_ref(src_f32, ref_f32):
        # map src luminance to have the same histogram as ref (both in 0..255)
        src_u8 = np.clip(src_f32, 0, 255).astype(np.uint8)
        ref_u8 = np.clip(ref_f32, 0, 255).astype(np.uint8)

        # CDFs
        src_hist = cv2.calcHist([src_u8], [0], None, [256], [0,256]).ravel()
        ref_hist = cv2.calcHist([ref_u8], [0], None, [256], [0,256]).ravel()
        src_cdf  = np.cumsum(src_hist) / (src_hist.sum() + 1e-6)
        ref_cdf  = np.cumsum(ref_hist) / (ref_hist.sum() + 1e-6)

        # For each src intensity, find ref intensity with closest CDF value
        mapping = np.interp(src_cdf, ref_cdf, np.arange(256))
        return mapping[src_u8].astype(np.float32)

    # After you compute Lmix (the blended L), do:
    Lmix = _hist_match_to_ref(Lmix, Ls)  # <-- replaces the mean/std normalization

    # Optional: darker edge reinforcement (subtle)
    if add_edges:
        e = cv2.Canny(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), edge_low, edge_high).astype(np.float32)
        e = cv2.GaussianBlur(e, (0,0), 0.8)
        Lmix = np.clip(Lmix - edge_weight * e, 0, 255)

    # Final gamma (1.0 = neutral)
    Lmix = np.clip(Lmix/255.0, 0, 1)
    Lmix = cv2.pow(Lmix, gamma) * 255.0
    Lmix_u8 = np.clip(Lmix, 0, 255).astype(np.uint8)

    lab_out = lab_src.copy()
    lab_out[...,0] = Lmix_u8
    out = cv2.cvtColor(lab_out, cv2.COLOR_LAB2BGR)
    cv2.imwrite(out_path, out)
    print("Saved:", out_path)

in_path  = "/content/out.png"
out_path = "/content/out_oil_natural.png"

oil_brush_balanced(
    in_path, out_path,
    brush_size=4, dyn_ratio=70, alpha=0.40,
    usm_radius=1.1, usm_amount=1.8,
    gamma=1.0,            # neutral; brightness controlled by hist match
    pre_smooth=False,     # keep detail
    add_edges=False
)

display(Image.open(out_path))

In [ ]:
im = Image.open(out_path)
w, h = im.size
print(f"Pixels: {w} × {h}")

for ppi in (300, 240, 200, 150):
    print(f"Max print at {ppi} PPI: {w/ppi:.2f} in × {h/ppi:.2f} in")